In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
!pip install simpletransformers
!pip install --upgrade simpletransformers
import re
import string
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, MaxPooling1D, Concatenate, Input, Flatten, Conv1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction
from gensim import models,corpora
from nltk.corpus import stopwords

In [ ]:
!pip install simpletransformers==0.48.0

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

In [ ]:
model = ClassificationModel('roberta', '../input/gurshaanbtpmodel/outputs/best_model/', num_labels=2)

In [ ]:
import json
f = open("../input/btpprecovid/Final_preCovid_data.json")
author_data = json.load(f)

In [ ]:
list_author = list(author_data.keys())
final_freq = dict()
final_freq['Author'] = []
final_freq['Dep Post'] = []
final_freq['Total Post'] = []

In [ ]:
count = 0;
test = pd.DataFrame()
for i in range(30000,len(list_author)):
    author = list_author[i]
    new_test = pd.DataFrame()
    new_test['text'] = author_data[author]
    aList = [author] * len(author_data[author])
    count += len(author_data[author])
    new_test['author'] = aList
    test = test.append(new_test, ignore_index=True)

In [ ]:
print(count)
print(len(test))

In [ ]:
data_test = test['text'].values

In [ ]:
predictions,raw = model.predict(data_test.tolist())

In [ ]:
test['pred'] = predictions

In [ ]:
test.head()

In [ ]:
ind = []
count = 0
for i in range(len(test)):
    s=test.iloc[i][0]
    if "[removed]" in s:
        ind.append(i)
        count+=1
test.drop(ind,inplace=True)
print(len(test))
print(count)
# make sure their sum is equal to total post you have predicted

In [ ]:
test=test.drop(columns=['text'])

In [ ]:
test.head()

In [ ]:
test.to_csv("30000-end.csv",encoding="utf-8-sig")